In [ ]:
pip install pds4_tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 KB 2.6 MB/s eta 0:00:00


In [ ]:
!unzip /content/drive/MyDrive/ISRO_OHRC_Images/ch2_ohr_ncp_20200827T0619368134_d_img_d18.zip -d /content/drive/MyDrive/unzipped_images

Archive:  /content/drive/MyDrive/ISRO_OHRC_Images/ch2_ohr_ncp_20200827T0619368134_d_img_d18.zip
   creating: /content/drive/MyDrive/unzipped_images/data/
   creating: /content/drive/MyDrive/unzipped_images/data/calibrated/
   creating: /content/drive/MyDrive/unzipped_images/data/calibrated/20200827/
  inflating: /content/drive/MyDrive/unzipped_images/data/calibrated/20200827/ch2_ohr_ncp_20200827T0619368134_d_img_d18.xml  
  inflating: /content/drive/MyDrive/unzipped_images/data/calibrated/20200827/ch2_ohr_ncp_20200827T0619368134_d_img_d18.img  
   creating: /content/drive/MyDrive/unzipped_images/miscellaneous/
   creating: /content/drive/MyDrive/unzipped_images/miscellaneous/calibrated/
   creating: /content/drive/MyDrive/unzipped_images/miscellaneous/calibrated/20200827/
  inflating: /content/drive/MyDrive/unzipped_images/miscellaneous/calibrated/20200827/ch2_ohr_ncp_20200827T0619368134_d_img_d18.oat  
  inflating: /content/drive/MyDrive/unzipped_images/miscellaneous/calibrated/202008

In [ ]:
import numpy as np
from scipy import ndimage
from scipy import signal
from numpy.lib.stride_tricks import as_strided
import cv2 
import os
from PIL import Image
import pds4_tools
from matplotlib import pyplot as plt
from numpy import asarray
from numpy import save

ModuleNotFoundError: ignored

In [ ]:
def block_mean(ar, fact):
    assert isinstance(fact, int), type(fact)
    sx, sy = ar.shape
    X, Y = np.ogrid[0:sx, 0:sy]
    regions = sy//fact * (X//fact) + Y//fact
    res = ndimage.mean(ar, labels=regions, index=np.arange(regions.max() + 1))
    res.shape = (sx//fact, sy//fact)
    return res

def apply_to_slices(func, a, axes, fact,radius):
  """Apply 'func' to each slice of array 'a', where a slice spans 'axes'.

  Args:
    func: function expecting an array of rank len(axes) and returning a
      modified array of the same shape.
    a: input array of arbitrary shape.
    axes: integer sequence specifying the slice orientation.
  """
  # The approach is to move the slice axes to the end of the array, reshape to
  # a 1-D array of slices, apply the user function to each slice, reshape back
  # to an outer array of slices, and finally move the slice axes back to their
  # original locations.  https://stackoverflow.com/a/61297133/
  assert len(axes) <= a.ndim
  outer_ndim = a.ndim - len(axes)
  a = np.moveaxis(a, axes, range(outer_ndim, a.ndim))
  outer_shape = a.shape[:outer_ndim]
  slice_shape = a.shape[outer_ndim:]
  a = a.reshape((-1,) + slice_shape)
  a = np.array([func(a_slice, fact,radius) for a_slice in a])
#   slice_shape[0] = slice_shape[0] // fact
#   slice_shape[1] = slice_shape[1] // fact
#   slice_shape1 = tuple(x // 2 for x in slice_shape)
  slice_shape1 = tuple(x // fact for x in slice_shape)
  a = a.reshape(outer_shape + slice_shape1)
  a = np.moveaxis(a, range(outer_ndim, a.ndim), axes)
  return a

def downsample_avg(image, redn_fact, radius):
    img_blur = ndimage.gaussian_filter(input = image, sigma = 0.3)
    img_down = block_mean(img_blur, redn_fact)
    return img_down

def downsample_all_avg(img_set,redn_fact,radius):
    img_set1=np.expand_dims(img_set,axis=3)
    applied_samp=apply_to_slices(downsample_avg,img_set1,axes=(1,2),fact=redn_fact, radius = radius)
    applied_samp=np.transpose(applied_samp,(0,3,1,2))
    applied_samp = np.squeeze(applied_samp, axis = 1)
    return applied_samp

def conv2d(a, b, s=1):
    Hout = (a.shape[1] - b.shape[0]) // s + 1
    Wout = (a.shape[2] - b.shape[1]) // s + 1
    Stride = (a.strides[0], a.strides[1] * s, a.strides[2] * s, a.strides[1], a.strides[2], a.strides[3])
    a = as_strided(a, (a.shape[0], Hout, Wout, b.shape[0], b.shape[1], a.shape[3]), Stride)
    return np.tensordot(a, b, axes=3)

def downsample_decimate(image, redn_fact, radius):
    image_blur = ndimage.gaussian_filter(input = image, sigma = 0.3)
    image_blur = np.expand_dims(image_blur, axis = 0)
    image_blur = np.expand_dims(image_blur, axis = 3)
    weight = np.zeros(redn_fact * redn_fact).astype('int')
    weight.shape = (redn_fact, redn_fact)
    weight[0][0] = 1
    weight = np.expand_dims(weight, axis = 2)
    weight = np.expand_dims(weight, axis = 3)
    stride = redn_fact
    convolved = conv2d(image_blur, weight, stride)
    convolved = np.squeeze(convolved, axis = 3)
    convolved = np.squeeze(convolved, axis = 0)
    return convolved

def downsample_all_decimate(img_set,redn_fact,radius):
    img_set1=np.expand_dims(img_set,axis=3)
    applied_samp=apply_to_slices(downsample_decimate,img_set1,axes=(1,2),fact=redn_fact, radius = radius)
    applied_samp=np.transpose(applied_samp,(0,3,1,2))
    applied_samp = np.squeeze(applied_samp, axis = 1)
    return applied_samp

In [ ]:
def read_ohrc(path):
    structure = pds4_tools.read(path)
    temp = structure[0].data  # matrix with all the grayscale values
    temp = np.array(temp)
    return temp

def reshape_split(image: np.ndarray, kernel_size: tuple):

    img_height, img_width = image.shape
    channels = 1
    tile_height, tile_width = kernel_size

    tiled_array = image.reshape(img_height // tile_height,
                                tile_height,
                                img_width // tile_width,
                                tile_width,
                                channels)
    tiled_array = tiled_array.swapaxes(1, 2)
    return tiled_array

def tiles1(image, tile_X, tile_Y):
    xtra_X = image.shape[0] - (image.shape[0] % tile_X)
    xtra_Y = image.shape[1] - (image.shape[1] % tile_Y)
    image = image[:xtra_X, :xtra_Y]
    tiled_images = reshape_split(image, (tile_X, tile_Y))
    tiled_images = np.squeeze(tiled_images, axis = 4)
    tiled_images = np.reshape(tiled_images, (tiled_images.shape[0] * tiled_images.shape[1], tiled_images.shape[2], tiled_images.shape[3]))
    return tiled_images

In [ ]:
def batchpasser(batch_size, tiled_images, redn_fact, radius, l):
#     idx = new_idx
    
    frames = tiled_images.shape[0]
    print("Creating Batches of Size:", batch_size)
    print("Total Batches found:", frames // batch_size + 1)
    for i in range(frames // batch_size + 1):
        print("Processing Batch", i + 1)
        if(i * batch_size + batch_size > frames):
            downsampled = downsample_all_decimate(tiled_images[i * batch_size : frames + 1, :, :], redn_fact, radius)
            for j in range(downsampled.shape[0]):
                if(redn_fact == 32):
                    save(r"C:\Users\acer\OneDrive - Indian Institute of Technology Patna\Desktop\ISRO\DownSampled\LR\\" + str(l[0]), downsampled[j])
                    l[0] += 1
                elif(redn_fact == 8):
                    save(r"C:\Users\acer\OneDrive - Indian Institute of Technology Patna\Desktop\ISRO\DownSampled\HR\\" + str(l[0]), downsampled[j])
                    l[0] += 1
            print("Saved Downsampled Images with Redn_Fact ({}), range({} to {})".format(redn_fact, i * batch_size, frames))
            break
        downsampled = downsample_all_decimate(tiled_images[i * batch_size : i * batch_size + batch_size, :, :], redn_fact, radius)
        for j in range(downsampled.shape[0]):
            if(redn_fact == 32):
                save(r"C:\Users\acer\OneDrive - Indian Institute of Technology Patna\Desktop\ISRO\DownSampled\LR\\" + str(l[0]), downsampled[j])
                l[0] += 1
            elif(redn_fact == 8):
                save(r"C:\Users\acer\OneDrive - Indian Institute of Technology Patna\Desktop\ISRO\DownSampled\HR\\" + str(l[0]), downsampled[j])
                l[0] += 1
        print("Saved Downsampled Images with Redn_Fact ({}), range({} to {})".format(redn_fact, i * batch_size, i * batch_size + batch_size - 1))
            # image is downsampled[j] -> 2d image
#     l[0] = idx

In [ ]:
def access_OHRC(paths, tile_X, tile_Y, redn_fact, batch_size, radius):
#     path where all OHRC folders are present is to be taken as i/p
    rootdir = paths
    directories = []
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            final = os.path.join(d, r'data\calibrated')
            for inside in os.listdir(final):
                direc = os.path.join(final, inside)
                for insider in os.listdir(direc):
                    if(insider[-3:] == "xml"):
                        direc = os.path.join(direc, insider)
                        directories.append(direc)
                        break
    l = [0]
    for direc in directories:
        print("Accessing directory:", direc)
        image = read_ohrc(direc)
        print("Image_Size =", image.shape[0], image.shape[1])
        print("Tiling Image with Tile_Size:", tile_X, tile_Y)
        tiled_image = tiles1(image, tile_X * redn_fact, tile_Y * redn_fact)
        print("Image Tiled... Total tiles found:", tiled_image.shape[0])
        print("Preparing Tiles for BatchPassing to DownSampler...")
        print("current_idx:", l[0])
        batchpasser(batch_size, tiled_image, redn_fact, radius, l)
#         del image
#         del tiled_image

In [ ]:
access_OHRC(r"C:\Users\acer\OneDrive - Indian Institute of Technology Patna\Desktop\ISRO\Data\\", 80, 80, 32, 4, 3)

In [ ]:
access_OHRC(r"C:\Users\acer\OneDrive - Indian Institute of Technology Patna\Desktop\ISRO\Data\\", 80, 80, 8, 4, 3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
